In [1]:
import pandas as pd

In [2]:
#df = pd.read_csv('song.csv')

In [3]:
n = 5  # every 5th line = 1% of the lines (because of to big, my laptop is crying :())
df = pd.read_csv("song.csv", header=0, skiprows=lambda i: i % n != 0)

In [4]:
a = []
b = []
c = []
for x, y, z in zip(df['song_id'], df['title'], df.index):
    if (x not in a) & (y not in b):
        a.append(x)
        b.append(y)
    if (x not in a) & (y in b):
        c.append(z)

In [5]:
c

[13624,
 13625,
 13626,
 13627,
 13628,
 13629,
 13630,
 13631,
 13632,
 13633,
 13634,
 13635,
 13636,
 13637,
 13638,
 13639,
 13640,
 13641,
 13642,
 13643,
 13644,
 13645,
 13646]

In [6]:
df.head()

,song_id,title,artist_name,user_id,rating
0,5076,Eli_ The Barrow Boy,The Decemberists,23987,1
1,5076,Eli_ The Barrow Boy,The Decemberists,51542,1
2,5076,Eli_ The Barrow Boy,The Decemberists,38547,1
3,5076,Eli_ The Barrow Boy,The Decemberists,879,1
4,5076,Eli_ The Barrow Boy,The Decemberists,17932,1


In [7]:
df.isna().sum()

song_id        0
title          0
artist_name    0
user_id        0
rating         0
dtype: int64

Buat dictionary:
song_id : original_title
original_title : song_id

### Check duplikat user review
Apakah ada user yang memberi review 2x pada song yang sama? Bila ada drop salah satu row nya.

In [8]:
df[df[['user_id', 'song_id']].duplicated()]

,song_id,title,artist_name,user_id,rating


In [9]:
df = df.drop(df[df[['user_id', 'song_id']].duplicated()].index).reset_index(drop=True)

In [10]:
song_dict1 = {}
song_dict2 = {}
for x, y in zip(df['song_id'], df['title']):
    if x not in song_dict1.keys():
        song_dict1[x] = y
        song_dict2[y] = x

In [11]:
df.head()

,song_id,title,artist_name,user_id,rating
0,5076,Eli_ The Barrow Boy,The Decemberists,23987,1
1,5076,Eli_ The Barrow Boy,The Decemberists,51542,1
2,5076,Eli_ The Barrow Boy,The Decemberists,38547,1
3,5076,Eli_ The Barrow Boy,The Decemberists,879,1
4,5076,Eli_ The Barrow Boy,The Decemberists,17932,1


Buat custom function rekomendasi song berdasarkan average rating tertinggi yang menerima 3 argumen:
1. jumlah song yang direkomendasikan
2. batas rating count untuk song yang dimasukan dalam rekomendasi
3. dataset (boleh ditransformasi)

Print rekomendasi 50 song yang telah diberi rating oleh minimal 100 orang.

In [12]:
df.groupby('song_id').count()['user_id']

song_id
16      134
28       19
44       15
45       23
98       22
       ... 
9873     11
9904     32
9921     20
9936     47
9956     18
Name: user_id, Length: 470, dtype: int64

In [13]:
df.groupby('song_id').mean()['rating']

song_id
16      1.164179
28      1.263158
44      1.000000
45      1.086957
98      1.090909
          ...   
9873    1.090909
9904    1.343750
9921    1.150000
9936    1.085106
9956    1.222222
Name: rating, Length: 470, dtype: float64

In [14]:
popular = pd.DataFrame({'count':df.groupby('song_id').count()['user_id'],
                       'rating':df.groupby('song_id').mean()['rating']})

In [15]:
popular.head()

,count,rating
song_id,,
16,134,1.164179
28,19,1.263158
44,15,1.000000
45,23,1.086957
98,22,1.090909


In [16]:
def popular_recom(k, count, dataset):
    a = []
    popular2 = dataset[dataset['count']>=count].sort_values('rating', ascending=False)
    for x in popular2.index[:k]:
        a.append(song_dict1[x])
    return a

In [17]:
popular_recom(50, 100, popular)

['Victoria (LP Version)',
 'Fuck Kitty',
 'Matricide',
 'Microdaze',
 'I Never Told You',
 'Helena (So Long & Goodnight) (Album Version)',
 'Californication (Album Version)',
 'Howlin\x19 For You',
 'Overboard',
 'Forgot About Dre',
 'Naive',
 'Fix You',
 'Mockingbird',
 'How You Remind Me',
 'Caring Is Creepy',
 'Empire State Of Mind (Part II) Broken Down',
 'More Than Words',
 'STARSTRUKK [FEATURINGKATYPERRY] (Explicit Bonus Version)',
 'Infinity',
 'Do You Remember',
 'Speed Of Sound',
 'Constellations',
 'Flume',
 'Mykonos',
 'Aerodynamic (Daft Punk Remix)',
 'Hearts On Fire',
 'Coda']

In [18]:
#pip install surprise

In [19]:
#conda install -c conda-forge scikit-surprise

In [20]:
from surprise import KNNBasic, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split, GridSearchCV

In [21]:
df2 = df[['user_id', 'song_id', 'rating']]

In [22]:
df2.head()

,user_id,song_id,rating
0,23987,5076,1
1,51542,5076,1
2,38547,5076,1
3,879,5076,1
4,17932,5076,1


In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17389 entries, 0 to 17388
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  17389 non-null  int64
 1   song_id  17389 non-null  int64
 2   rating   17389 non-null  int64
dtypes: int64(3)
memory usage: 407.7 KB


### Splitting Train (80%) and Test Dataset (20%)

In [24]:
reader = Reader(rating_scale=(0,5))
data = Dataset.load_from_df(df2, reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [25]:
param_grid = {'k': [20, 30, 40],
             'min_k': [3, 6, 9],
             'sim_options' : {'name': ['msd', 'pearson', 'cosine'],
                             'user_based': [True]}}
gscv = GridSearchCV(KNNBasic, param_grid, cv=4, measures = ['rmse'])

In [26]:
gscv.fit(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd simil

In [27]:
gscv.best_params

{'rmse': {'k': 20,
  'min_k': 3,
  'sim_options': {'name': 'pearson', 'user_based': True}}}

In [28]:
gscv.best_score

{'rmse': 0.49066105007470284}

In [29]:
knn_user = KNNBasic(**gscv.best_params)
knn_user.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [30]:
prediction = knn_user.test(testset)
accuracy.rmse(prediction)

RMSE: 0.4879


0.48791818695903694

buat list song yang pernah diberi review oleh user 4, dan buat list song yang belum pernah diberi review oleh user 4

In [31]:
b = []
c = []
for x in df2['song_id'].unique():
    if x in df2[df2['user_id']==4]['song_id'].unique():
        b.append(x)
    else:
        c.append(x)

In [32]:
knn_user.predict(4,1371, r_ui=4, verbose=True)

user: 4          item: 1371       r_ui = 4.00   est = 1.19   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid=4, iid=1371, r_ui=4, est=1.1912155847890158, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [33]:
knn_user.predict(4,1405, verbose=True)

user: 4          item: 1405       r_ui = None   est = 1.19   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}


Prediction(uid=4, iid=1405, r_ui=None, est=1.1912155847890158, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [34]:
knn_user.predict(4,1405).est

1.1912155847890158

Fungsi custom untuk mendapatkan rekomendasi berdasarkan collaborative filtering user-user

1. dataset
2. user_id
3. k
4. algo

Print top 10 rekomendasi song untuk user 4 dengan algoritma user-user (nama song+estimasi rating))

In [43]:
df2.head()

,user_id,song_id,rating
0,23987,5076,1
1,51542,5076,1
2,38547,5076,1
3,879,5076,1
4,17932,5076,1


In [35]:
def get_recom_user(dataset, user, k, algo):
    dataset_user = pd.pivot_table(dataset, index='user_id', columns='song_id').loc[user]
    dataset_user_null = dataset_user[dataset_user.isnull()].reset_index()
    non_interacted = []
    for x,y in zip(dataset_user_null.loc[:,user], dataset_user_null.loc[:,'song_id']):
        if pd.isnull(x) == True:
            non_interacted.append(y)
    non_interacted_df = {'song':{}, 'estimated_rating':{}}
    for x, y in zip(non_interacted, range(len(non_interacted))):
        non_interacted_df['song'][y] = song_dict1[x]
        non_interacted_df['estimated_rating'][y] = algo.predict(user, x).est
    f = pd.DataFrame(non_interacted_df).sort_values('estimated_rating', ascending=False).iloc[:k,:]
    return f

In [50]:
get_recom_user(df2, 10, 20, knn_user)

,song,estimated_rating
252,The Handshake,5.000000
384,Naughty Girl,2.000000
75,Army of Me,2.000000
337,Victoria (LP Version),2.000000
106,Cult Status,2.000000
97,One Of The Boys,2.000000
191,On Bended Knee,2.000000
89,Drugs,2.000000
195,It's My Life,2.000000
396,Fix You,1.750000
